In [ ]:
train_sessions = pd.DataFrame()
chunks = pd.read_json(data_path / 'train.jsonl', lines=True, chunksize=100_000)

for e, chunk in enumerate(chunks):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    if e < 2:
        for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
            for event in events:
                event_dict['session'].append(session)
                event_dict['aid'].append(event['aid'])
                event_dict['ts'].append(event['ts'])
                event_dict['type'].append(event['type'])
        chunk_session = pd.DataFrame(event_dict)
        train_sessions = pd.concat([train_sessions, chunk_session])
    else:
        break
        
train_sessions = train_sessions.reset_index(drop=True)


train_sessions['ts'] = pd.to_datetime(train_sessions['ts'], unit='ms')

train_sessions['date'] = train_sessions['ts'].dt.date

df_aug = train_sessions[train_sessions['date'] == pd.to_datetime('2022-08-01')]

df_orders = train_sessions[train_sessions['type'] == 'orders']

In [ ]:
item_freq = df_orders['aid'].value_counts()
user_freq = df_orders['session'].value_counts()

items = item_freq[item_freq >= 10].index
users = user_freq[user_freq >= 10].index

filtered_df = df_orders[df_orders['aid'].isin(items) & df_orders['session'].isin(users)]

In [ ]:
filtered_by_session = filtered_df.groupby('session').filter(lambda group: (group['type'] == 'orders').sum() > 5)

final_df = filtered_df.groupby('aid').filter(lambda group: len(group) > 5)

In [ ]:
final_df.reset_index(drop=True)

In [ ]:
freq = final_df.groupby(['session', 'aid']).size().reset_index(name='frequency')

final_df = final_df.merge(freq, on=['session', 'aid'], how='left')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(final_df['frequency'], kde=True, bins=30)

plt.title('Distribution of frequency')
plt.xlabel('Frequency')
plt.ylabel('Density')

plt.show()

In [ ]:
G = nx.Graph()

for index, row in df.iterrows():
    G.add_node(row['session'], type='user')
    G.add_node(row['aid'], type='item')
    
    G.add_edge(row['session'], row['aid'], weight=row['stars'])